In [1]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.get_device_name(0))  # Should display your GPU name

2.5.1+cu118
True
NVIDIA GeForce RTX 3060


Q1. 1. Draw Computation Graph and work out the gradient dz/da  by following the path 
back from z to a and compare the result with the analytical gradient.   
x = 2*a + 3*b 
y = 5*a*a + 3*b*b*b 
z = 2*x + 3*y

In [36]:
import numpy as np
import math

In [6]:
a = torch.tensor(2.0, requires_grad=True)
b = torch.tensor(4.0, requires_grad=True) 
x =  2*a +3*b
y = 5*a*a + 3*b*b*b
z = 2*x +3*y
z.backward()
print("gradient at a: ", a.grad)


gradient at a:  tensor(64.)


q2) For the following Computation Graph, work out the gradient da/dw  by following the 
path back from a to w and compare the result with the analytical gradient. 

In [23]:
x = torch.tensor(2.0, requires_grad=True)
b = torch.tensor(4.0, requires_grad=True) 
w = torch.tensor(6.0, requires_grad=True) 
u = w*x
v=(u+b) *1
if max(v,0) !=0:
    a =v
else:
    a = torch.tensor(0.0, requires_grad=True) 
#a = torch.relu(v)
print(a)
a.backward()
print("gradient at w: ", w.grad)

tensor(16., grad_fn=<MulBackward0>)
gradient at w:  tensor(2.)


In [29]:

def manual_q2(x,w,b):
    u = w*x
    v=(u+b) *1
    if max(v,0) !=0:
        a =v
        dadv = 1
    else:
        a = torch.tensor(0.0, requires_grad=True)
        dadv= 0
    dvdu=1
    dvdb =1
    dudw=x
    dudx = w
    temp = dadv*dvdu*dudw
    return temp
print("analytical sol: ", manual_q2(2,4,6))
    


analytical sol:  2


3. Repeat the Problem 2 using Sigmoid function

In [40]:
x = torch.tensor(0.20, requires_grad=True)
w = torch.tensor(0.40, requires_grad=True) 
b = torch.tensor(0.60, requires_grad=True) 
u = w*x
v=(u+b) *1
a =1/(1+torch.exp(-1*v))
#a = torch.sigmoid(v)
print(a)
a.backward()
print("gradient at w: ", w.grad)

tensor(0.6637, grad_fn=<MulBackward0>)
gradient at w:  tensor(0.0446)


In [39]:
def manual_q3(x,w,b):
    u = w*x
    v=(u+b) *1
    a = 1 / (1 + math.exp(-v))
    dadv = a*(1-a)
    dvdu=1
    dvdb =1
    dudw=x
    dudx = w
    temp = dadv*dvdu*dudw
    return temp
print("analytical sol: ", manual_q3(0.2,0.4,0.6))

analytical sol:  0.04463792779446517


4. Verify that the gradients provided by PyTorch match with the analytical gradients of 
the function f= exp(-x2-2x-sin(x))  w.r.t x 

In [51]:
x = torch.tensor(2.0, requires_grad=True)
a = torch.exp((-1*x*x)-(2*x)-torch.sin(x))
print(a)
a.backward()
print("gradient at x: ", x.grad.item())

tensor(0.0001, grad_fn=<ExpBackward0>)
gradient at x:  -0.0007545282132923603


In [52]:
def manual_q4(x):
    u = -1*x*x
    v= -2*x
    p = math.sin(x)
    q= u+v-p
    a = math.exp(q)
    dadq = math.exp(q)
    dqdu=1
    dqdv=1
    dqdp=-1
    dudx = -2*x
    dvdx=-2
    dpdx=math.cos(x)
    
    temp = (dadq*dqdu*dudx)+(dadq*dqdv*dvdx)+(dadq*dqdp*dpdx)
    return temp
print("analytical sol: ", manual_q4(2.0))

analytical sol:  -0.0007545278582400674


5. Compute gradient for the function y=8x4+ 3x3 +7x2+6x+3 and verify the gradients 
provided by PyTorch with the analytical gradients. A snapshot of the Python code is 
provided below. 

In [53]:
x = torch.tensor(2.0, requires_grad=True)
a = 8*x**4 + 3*x**3 + 7*x**2 + 6*x +3
print(a)
a.backward()
print("gradient at x: ", x.grad.item())

tensor(195., grad_fn=<AddBackward0>)
gradient at x:  326.0


In [54]:
def manual_q4(x):
    u = 8*x**4
    v= 3*x**3
    p = 7*x**2
    q= 6*x
    a = u+v+p+q+3
    dadq = 1
    dadu=1
    dadv=1
    dadp=1
    dqdx=6
    dpdx=14*x
    dvdx=9*x**2
    dudx = 32*x**3
    
    temp = (dadq*dqdx)+(dadv*dvdx)+(dadp*dpdx)+(dadu*dudx)
    return temp
print("analytical sol: ", manual_q4(2.0))

analytical sol:  326.0


6. For the following function, computation graph is provided below.

In [63]:
x = torch.tensor(2.6, requires_grad=True)
y = torch.tensor(6.5, requires_grad=True) 
z = torch.tensor(2.26, requires_grad=True)
a =  2*x
a.retain_grad()
b = torch.sin(y)
c = a/b
d = c*z
e = torch.log(d+1)
f = torch.tanh(e)
f.backward()
print("gradient at y: ", y.grad.item())
print("gradient at a: ", a.grad.item())

gradient at y:  -0.005758530460298061
gradient at a:  0.00024393710191361606


In [67]:
def calc_grad(x, y, z, a, b, c, d, e, f):
    dfde = 1-torch.tanh(e)**2
    dedd = 1/(d+1); dddc = z
    dcdb = -1*a/b**2
    dbdy = torch.cos(y)
    dfdy = dfde*dedd*dddc*dcdb*dbdy
    return dfdy.item()
print("analytical solution: ", calc_grad(x,y,z,a,b,c,d,e,f))

analytical solution:  -0.005758530460298061
